In [2]:
import pandas as pd
import os


In [9]:
for file in os.listdir(input_dir):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(input_dir, file))
        print(f"{file} ➜ 欄位：{df.columns.tolist()}")

6.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
7.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
5.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
4.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
1.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
3.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
2.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
23.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
22.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
36.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
20.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']
34.csv ➜ 欄位：['no', 'control', 'stimulus', 'position', 'ui', 'response', 'rt', 'SR']

In [11]:

for file in os.listdir(input_dir):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(input_dir, file))
        subject_id = file.replace('.csv', '')

        ### ✅ 分析 1：control + stimulus + ui 組合分析
        group_cols_1 = ['control', 'stimulus', 'ui', 'SR']
        cond_stats = (
            df.groupby(group_cols_1).size()
            .unstack()
            .fillna(0)
            .astype(int)
            .reset_index()
        )
        cond_stats.columns.name = None
        cond_stats['SR_0_count'] = cond_stats.get(0, 0)
        cond_stats['SR_1_count'] = cond_stats.get(1, 0)
        cond_stats.drop(columns=[0, 1], errors='ignore', inplace=True)
        cond_stats['total'] = cond_stats['SR_0_count'] + cond_stats['SR_1_count']
        cond_stats['SR_0_ratio'] = cond_stats['SR_0_count'] / cond_stats['total']
        cond_stats['SR_1_ratio'] = cond_stats['SR_1_count'] / cond_stats['total']

        ### ✅ 分析 2：只看 control 分析
        group_cols_2 = ['control', 'SR']
        control_stats = (
            df.groupby(group_cols_2).size()
            .unstack()
            .fillna(0)
            .astype(int)
            .reset_index()
        )
        control_stats.columns.name = None
        control_stats['SR_0_count'] = control_stats.get(0, 0)
        control_stats['SR_1_count'] = control_stats.get(1, 0)
        control_stats.drop(columns=[0, 1], errors='ignore', inplace=True)
        control_stats['total'] = control_stats['SR_0_count'] + control_stats['SR_1_count']
        control_stats['SR_0_ratio'] = control_stats['SR_0_count'] / control_stats['total']
        control_stats['SR_1_ratio'] = control_stats['SR_1_count'] / control_stats['total']

        ### ✅ 輸出 CSV（兩份）
        cond_stats.to_csv(os.path.join(output_dir, f'{subject_id}_condition_combo.csv'), index=False, encoding='utf-8-sig')
        control_stats.to_csv(os.path.join(output_dir, f'{subject_id}_control_only.csv'), index=False, encoding='utf-8-sig')

        print(f"✅ 完成分析：{subject_id}")

✅ 完成分析：6
✅ 完成分析：7
✅ 完成分析：5
✅ 完成分析：4
✅ 完成分析：1
✅ 完成分析：3
✅ 完成分析：2
✅ 完成分析：23
✅ 完成分析：22
✅ 完成分析：36
✅ 完成分析：20
✅ 完成分析：34
✅ 完成分析：35
✅ 完成分析：21
✅ 完成分析：25
✅ 完成分析：31
✅ 完成分析：19
✅ 完成分析：18
✅ 完成分析：30
✅ 完成分析：24
✅ 完成分析：32
✅ 完成分析：26
✅ 完成分析：27
✅ 完成分析：33
✅ 完成分析：16
✅ 完成分析：17
✅ 完成分析：29
✅ 完成分析：15
✅ 完成分析：14
✅ 完成分析：28
✅ 完成分析：10
✅ 完成分析：11
✅ 完成分析：13
✅ 完成分析：12
✅ 完成分析：9
✅ 完成分析：8


In [18]:

input_dir = './byno'
output_path = './results_per_subject/summary.csv'
os.makedirs('./results_per_subject', exist_ok=True)

summary_rows = []

for file in os.listdir(input_dir):
    if file.lower().endswith('.csv'):
        df = pd.read_csv(os.path.join(input_dir, file))
        df.columns = df.columns.str.strip()  # 保險清除欄位空格

        subject_id = file.replace('.csv', '')

        # 統計 SR 分布
        sr_counts = df['SR'].value_counts().to_dict()
        sr_0 = sr_counts.get(0, 0)
        sr_1 = sr_counts.get(1, 0)
        total = sr_0 + sr_1

        # 計算比例
        sr_0_ratio = sr_0 / total if total else 0
        sr_1_ratio = sr_1 / total if total else 0

        summary_rows.append({
            'no': subject_id,
            'SR_0_count': sr_0,
            'SR_1_count': sr_1,
            'total': total,
            'SR_0_ratio': round(sr_0_ratio, 3),
            'SR_1_ratio': round(sr_1_ratio, 3)
        })

# 整理成表並輸出
summary_df = pd.DataFrame(summary_rows)
summary_df = summary_df.sort_values(by='no')
summary_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("✅ 所有受試者 SR 整體比例已輸出至：", output_path)

✅ 所有受試者 SR 整體比例已輸出至： ./results_per_subject/summary.csv
